In [1]:
import cv2 as cv
import numpy as np
import time
import os
import mediapipe as mp

In [2]:
# Hand Traching Module
class handDetector():
    def __init__(self , mode = False , maxHands = 2 , detectionCon = 0.5 , trackCon = 0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.trackCon = trackCon
        
        self.mpHands= mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode , self.maxHands ,1, self.detectionCon , self.trackCon)
        self.mpDraw = mp.solutions.drawing_utils
        
        self.tipIds = [4 , 8 , 12 , 16 , 20]
        
    def findHands(self,img,draw = True):
        imgRGB =  cv.cvtColor(img , cv.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw : self.mpDraw.draw_landmarks(img,handLms , self.mpHands.HAND_CONNECTIONS)
        return img
    
    def findPosition(self , img , handNo = 0 , draw = True):
        self.lmList = []
        if self.results.multi_hand_landmarks:
            
            myHand = self.results.multi_hand_landmarks[handNo]
            
            for id , lm in enumerate(myHand.landmark):
                    h, w ,c = img.shape
                    cx , cy = int(lm.x*w ) , int(lm.y * h)
                    self.lmList.append([id, cx, cy])
                    
                    if draw : cv.circle(img,(cx , cy) , 25 , (255,0,255) , cv.FILLED)
                    
        return self.lmList
    
    def fingersUp(self):
        fingers = []
        
        # thums
        if self.lmList[self.tipIds[0]][1] < self.lmList[self.tipIds[0] - 1][1]:
            fingers.append(0)
        else:
            fingers.append(1)
            
        # Fingers
        for id in range(1,5):
            if self.lmList[self.tipIds[id]][2] < self.lmList[self.tipIds[id]-2][2]:
                fingers.append(1)
            else:
                fingers.append(0)
                
        return fingers
        
        

In [3]:
folderPath = 'headers'
myList = os.listdir(folderPath)
print(myList)

overlayList = []

for imgPath in myList:
    image = cv.imread(f"{folderPath}/{imgPath}")
    overlayList.append(image)

['blue.png', 'erser.png', 'green.png', 'red.png']


In [11]:
header = overlayList[3]
drawColor = (0, 0 ,255)
brushThickness = 15
eraserThickness = 50

cap = cv.VideoCapture(0)
cap.set(3 , 1280 ) # Width
cap.set(4 , 720 ) # Height

detector = handDetector(detectionCon=0.85)
xp , yp = 0,0
canvas = np.zeros((720 , 1280 , 3), np.uint8)

while cap.isOpened():
    
    # Preparing frame
    _ , frame = cap.read()
    frame = cv.flip(frame , 1) # Fliping the image
    
    # Find hand landmarks
    frame = detector.findHands(frame) # Find hand
    lmList = detector.findPosition(frame , draw= False)
    
    if len(lmList) != 0:
        
        # Tip and Middle Finger
        x1 , y1 = lmList[8][1:]
        x2 , y2 = lmList[12][1:]
    
        # Check Up fingers
        fingers = detector.fingersUp()
        
        # Selection Mode
        if fingers[1] and fingers[2]:
            print("Selection Mode")
            xp , yp = 0,0
            
            if y1 < 125 :
                 
                # BGR
                # RED
                if 250 < x1 < 450:
                    header = overlayList[3]
                    drawColor = (0,0,255)
                # BLUE
                elif 550 < x1 < 750:
                    header = overlayList[0]
                    drawColor = (255,125,0)
                # GREEN
                elif 800 < x1 < 950:
                    header = overlayList[2]
                    drawColor = (0,255,0)
                # Erser
                elif 1000 < x1 < 1200:
                    header = overlayList[1]
                    drawColor = (0,0,0)
            cv.rectangle(frame , (x1 , y1 - 25) , (x2 , y2 + 25) , drawColor , cv.FILLED)
                    

        # Drawing Mode
        if fingers[1] and fingers[2] == False:
            cv.circle(frame , (x1 , y1) , 15 , drawColor , cv.FILLED)
            print("Drawing Mode")
            if xp == 0 and yp == 0:
                xp , yp = x1 , y1
            if drawColor == (0,0,0):
                cv.line(frame , (xp , yp) , (x1 ,y1) , drawColor , eraserThickness)
                cv.line(canvas , (xp , yp) , (x1 ,y1) , drawColor , eraserThickness)
            else:
                cv.line(frame , (xp , yp) , (x1 ,y1) , drawColor , brushThickness)
                cv.line(canvas , (xp , yp) , (x1 ,y1) , drawColor , brushThickness)
        
            xp , yp = x1 , y1
            
    gray = cv.cvtColor(canvas , cv.COLOR_BGR2GRAY)
    _, inv = cv.threshold(gray , 50 , 255 ,cv.THRESH_BINARY_INV)    
    inv = cv.cvtColor(inv , cv.COLOR_GRAY2BGR)
    frame = cv.bitwise_and(frame , inv)
    frame = cv.bitwise_or(frame , canvas)
            
    
    # Adding headers
    frame[0:125 , 0:1280 ] = header
    # frame = cv.addWeighted(frame, 0.5,canvas , 0.5,0)
    
    cv.imshow("Capturing",frame)
    cv.imshow("Canvas",canvas)
    
    # Exit
    if cv.waitKey(1) == ord('q'):
        break
    
cap.release()    
cv.destroyAllWindows()    

Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Drawing Mode
Drawing Mode
Drawing Mode
Drawing Mode
Drawing Mode
Drawing Mode
Drawing Mode
Drawing Mode
Drawing Mode
Drawing Mode
Drawing Mode
Drawing Mode
Drawing Mode
Drawing Mode
Drawing Mode
Drawing Mode
Drawing Mode
Drawing Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Selection Mode
Drawing Mode
Drawing Mode
Drawing Mode
Drawing Mode
Drawing Mode
Drawing Mode
Drawing Mode
Drawing Mode
Drawing Mode
Draw